In [17]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [18]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

In [20]:
random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist=model.fit(trainX, trainY, epochs=100, batch_size=15, verbose=1)
model.save('chatbot_model.h5',hist)
print('Done')

Epoch 1/100
2/2 [==============================] - 1s 4ms/step - loss: 1.6066 - accuracy: 0.2500
Epoch 2/100
2/2 [==============================] - 0s 4ms/step - loss: 1.6066 - accuracy: 0.3929
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 1.5563 - accuracy: 0.4643
Epoch 4/100
2/2 [==============================] - 0s 4ms/step - loss: 1.5698 - accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 4ms/step - loss: 1.5192 - accuracy: 0.4643
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 1.5356 - accuracy: 0.4643
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 1.4610 - accuracy: 0.6786
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 1.4453 - accuracy: 0.5000
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 1.4491 - accuracy: 0.4286
Epoch 10/100
2/2 [==============================] - 0s 4ms/step - loss: 1.4401 - accuracy: 0.5357
Epoch 11/100
2/2 [===========

2/2 [==============================] - 0s 3ms/step - loss: 0.2430 - accuracy: 0.9643
Epoch 85/100
2/2 [==============================] - 0s 3ms/step - loss: 0.2502 - accuracy: 0.9286
Epoch 86/100
2/2 [==============================] - 0s 3ms/step - loss: 0.2515 - accuracy: 0.9643
Epoch 87/100
2/2 [==============================] - 0s 3ms/step - loss: 0.2711 - accuracy: 0.9286
Epoch 88/100
2/2 [==============================] - 0s 3ms/step - loss: 0.2405 - accuracy: 0.9643
Epoch 89/100
2/2 [==============================] - 0s 2ms/step - loss: 0.2301 - accuracy: 0.9643
Epoch 90/100
2/2 [==============================] - 0s 4ms/step - loss: 0.1789 - accuracy: 0.9643
Epoch 91/100
2/2 [==============================] - 0s 3ms/step - loss: 0.3190 - accuracy: 0.9286
Epoch 92/100
2/2 [==============================] - 0s 3ms/step - loss: 0.2187 - accuracy: 0.9286
Epoch 93/100
2/2 [==============================] - 0s 4ms/step - loss: 0.2877 - accuracy: 0.8929
Epoch 94/100
2/2 [===============